In [84]:
import pandas as pd
import pickle as pkl
import numpy as np
import os, gc, math, sys, re
import os,sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import *
from datetime import datetime as dt
from datetime import datetime, timedelta
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from sklearn.cluster import KMeans
import time
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import holidays as h

In [66]:
def dummie_train(df, col):
    uniq = df[col].unique()
    for i in uniq:
        name = col + '_'+ str(i)
        df.loc[:,name] = df[col] == i
        df.loc[:,name] = df.loc[:,name].astype(int)
    return df, uniq
def dummie_test(df, col, uniq):
    for i in uniq:
        name = col + '_'+ str(i)
        df.loc[:,name] = df[col] == i
        df.loc[:,name] = df.loc[:,name].astype(int)
    return df

In [44]:
def show_nulls(data):
    d = ((c, data[c][pd.isnull(data[c])].shape[0]) for c in data.columns)
    return pd.DataFrame(d, columns=['name', 'nans'])

In [45]:
df = pd.read_pickle('../df.pkl')

In [46]:
df_cancelled = df[pd.isnull(df.Begin)]
df = df[~pd.isnull(df.Begin)]
print df.shape[0], df_cancelled.shape[0]

27424 1580


_____

In [48]:
print list(df.columns)

[u'Scheduled', u'Begin', u'Complete', u'Scan start', u'PID', u'EID', u'Arrive', u'Modality', u'Exam']


#### Convert time

In [49]:
unix_date = datetime(2010, 1, 1,0,0,0)
td = timedelta(seconds=1)
for i in tqdm(['Scheduled', 'Arrive', 'Begin','Scan start', 'Complete']):
    df[i+'_UNIX'] = list(map(lambda x: (x - unix_date) /  td, df[i]))

100%|██████████| 5/5 [00:04<00:00,  1.19it/s]


In [50]:
day = datetime.strptime('02.10.10','%d.%m.%y') - datetime.strptime('01.10.10','%d.%m.%y')
week = datetime.strptime('17.10.10 10:10:10','%d.%m.%y %H:%M:%S') - datetime.strptime('10.10.10 10:10:10','%d.%m.%y %H:%M:%S')
month = datetime.strptime('01.10.10','%d.%m.%y') - datetime.strptime('01.09.10','%d.%m.%y')

In [51]:
# время ожидания от прихода

# TARGET

def time_of_expected(df):
    df['time_of_expected'] = df.Begin - df.Arrive
    df['time_of_expected'] = list(map(lambda x: x.delta/1000000000,tqdm(df['time_of_expected'])))
    return df

In [52]:
# добавить дамии на месяц (но сейчас у нас эта штука не будет иметь значение (по идее) 
# так как мы используем как тест один обособленный месяц)
df['month'] = list(map(lambda x: x.month, df.Arrive))

In [53]:
# добавить дамми на день недели
df['day_of_week'] = list(map(lambda x: datetime.weekday(x), df.Arrive))

In [54]:
# изменить года для нового датасета
us_hol = h.US(state='MA', years=[2013, 2014, 2015, 2016,])

# добавить расстояние до ближайщего справа праздника
def right(x, hol = us_hol):
    days = []
    for i in us_hol:
        days.append((datetime.date(x)-i).days)
    day = np.min(pd.Series(days)[pd.Series(days) >= 0])
    return day
df['right_holiday'] = list(map(lambda x: right(x), tqdm(df.Arrive)))
# добавить расстояние до ближайщего слева праздника
def left(x):
    days = []
    for i in us_hol:
        days.append((datetime.date(x)-i).days)
    day = np.max(pd.Series(days)[pd.Series(days) <= 0])
    return day
df['left_holiday'] = list(map(lambda x: left(x), tqdm(df.Arrive)))
                          
df['is_holiday'] = list(map(lambda x: int(x in us_hol), df.Arrive))

100%|██████████| 27424/27424 [00:33<00:00, 813.47it/s]


In [55]:
# время ожидание от запланированного приема (может быть отрицательным)

def time_of_expected_on_shedule(df):
    df['time_of_expected_on_shedule'] = df.Begin - df.Scheduled
    df['time_of_expected_on_shedule'] = list(map(lambda x: x.delta/1000000000,df['time_of_expected_on_shedule']))
    return df

In [56]:
# сколько выполнялось исследование (может быть оооочень большим)

def time_of_procedure(df):
    df['time_of_procedure'] = df['Complete'] - df['Begin']
    df['time_of_procedure'] = list(map(lambda x: x.delta/1000000000,df['time_of_procedure']))
    return df

In [57]:
# сколько пациентов находилось в очереди за delta секунд

def lq(x,y,delta = 0, df = df):
    return df[(df.Arrive_UNIX < x-delta) & (df.Begin_UNIX > x-delta) & (df.Modality == y)].shape[0]
def len_of_queque(df, delta = 0):
    df['len_of_queque_before_'+str(delta)+'_seconds'] = list(map(lambda x, y: lq(x,y,delta,df) , df.Arrive_UNIX, df.Modality))
    return df

In [58]:
# количество зашедших на исследование за некоторое время

# time_delta в секундах

def come_in(x,df,time_delta):
    return df[(df.Begin_UNIX >= ((x - unix_date)/td - time_delta) ) & (df.Begin_UNIX <= (x - unix_date)/td )].shape[0]
def count_who_come_in(df,time_delta):
    df['count_who_come_in_previous_'+str(time_delta)+'_seconds'] = list(map(lambda x, y: come_in(x,df[df.Modality == y],time_delta) , df.Arrive, df.Modality))
    return df

In [59]:
# количество вышедших с исследования за некоторое время

# time_delta в секундах

def come_out(x,df,time_delta):
    return df[(df.Complete_UNIX >= ((x - unix_date)/td - time_delta) ) & (df.Complete_UNIX <= (x - unix_date)/td )].shape[0]
def count_who_come_out(df,time_delta):
    df['count_who_come_out_previous_'+str(time_delta)+'_seconds'] = list(map(lambda x, y: come_out(x,df[df.Modality == y],time_delta) , df.Arrive, df.Modality))
    return df

In [60]:
# количество пришедших в госпиталь за некоторое время

# time_delta в секундах

def arrive_in(x,df,time_delta):
    return df[(df.Arrive_UNIX >= ((x - unix_date)/td - time_delta) ) & (df.Arrive_UNIX <= (x - unix_date)/td )].shape[0]

def count_who_arrive_in(df,time_delta):
    df['count_who_arrive_in_previous_'+str(time_delta)+'_seconds'] = list(map(lambda x, y: 
            arrive_in(x,df[df.Modality == y],time_delta) , df.Arrive, df.Modality))
    return df

In [61]:
# среднее время ожидания по всем людям, которые пришли за некоторый промежуток времени

def mean_time(x,df, period):
    return np.mean(df[(df.Arrive < x) & (df.Arrive >= x - period)].time_of_expected)
def mean_time_of_exp(df,multiply = 1,period = day):
    # period as time in unix
    if period == day:
        name = 'day'
    elif period == week:
        name = 'week'
    else:
        name = 'month'
    df['mean_time_of_exp_'+str(multiply)+'_'+name] = list(map(lambda x,y: mean_time(x,df[df.Modality == y], period*multiply), df.Arrive, df.Modality))
    return df

# оклонение по всем людям, которые пришли за некоторый промежуток времени

def std_time(x,df, period):
    return np.std(df[(df.Arrive < x) & (df.Arrive >= x - period)].time_of_expected)
def std_time_of_exp(df,multiply = 1,period = day):
    # period as time in unix
    if period == day:
        name = 'day'
    elif period == week:
        name = 'week'
    else:
        name = 'month'
    df['std_time_of_exp_'+str(multiply)+'_'+name] = list(map(lambda x,y: std_time(x,df[df.Modality == y], period*multiply), df.Arrive, df.Modality))
    return df

In [62]:
# среднее время проведения исследования по всем людям, которые пришли за некоторый промежуток времени

def mean_time_2(x,df, period):
    return np.mean(df[(df.Arrive < x) & (df.Arrive >= x - period)].time_of_procedure)
def mean_time_of_proced(df,multiply = 1,period = day):
    # period as time in unix
    if period == day:
        name = 'day'
    elif period == week:
        name = 'week'
    else:
        name = 'month'
    df['mean_time_of_proced_'+str(multiply)+'_'+name] = list(map(lambda x,y: mean_time_2(x,df[df.Modality == y], period*multiply), df.Arrive, df.Modality))
    return df

# оклонение проведения исследования по всем людям, которые пришли за некоторый промежуток времени

def std_time_2(x,df, period):
    return np.std(df[(df.Arrive < x) & (df.Arrive >= x - period)].time_of_procedure)
def std_time_of_proced(df,multiply = 1,period = day):
    # period as time in unix
    if period == day:
        name = 'day'
    elif period == week:
        name = 'week'
    else:
        name = 'month'
    df['std_time_of_proced_'+str(multiply)+'_'+name] = list(map(lambda x,y: std_time_2(x,df[df.Modality == y], period*multiply), df.Arrive, df.Modality))
    return df

In [63]:
# среднее время отклонения от расписания по всем людям, которые пришли за некоторый промежуток времени

def mean_time_3(x,df, period):
    return np.mean(df[(df.Arrive < x) & (df.Arrive >= x - period)].time_of_expected_on_shedule)
def mean_time_of_exp_schel(df,multiply = 1,period = day):
    # period as time in unix
    if period == day:
        name = 'day'
    elif period == week:
        name = 'week'
    else:
        name = 'month'
    df['mean_time_of_exp_schel_'+str(multiply)+'_'+name] = list(map(lambda x,y: mean_time_3(x,df[df.Modality == y], period*multiply), df.Arrive, df.Modality))
    return df

# оклонение отклонения от расписания по всем людям, которые пришли за некоторый промежуток времени

def std_time_3(x,df, period):
    return np.std(df[(df.Arrive < x) & (df.Arrive >= x - period)].time_of_expected_on_shedule)
def std_time_of_exp_schel(df,multiply = 1,period = day):
    # period as time in unix
    if period == day:
        name = 'day'
    elif period == week:
        name = 'week'
    else:
        name = 'month'
    df['std_time_of_exp_schel_'+str(multiply)+'_'+name] = list(map(lambda x,y: std_time_3(x,df[df.Modality == y], period*multiply), df.Arrive, df.Modality))
    return df

In [ ]:
# чувак с какой модальностью сейчас на исследовании

def closest_modality(df):
    return modality
def now_with_doctor(df):
    df['now_with_doctor'] = list(map(lambda x,y: closest_modality(df[(df.Modality == y) & (df.Begin_UNIX < x)]), df.Begin_UNIX, df.Modality ))
    return df

def has_cancelled_today(df):
    df['has_cancelled_today'] = map(lambda x,y: 
                                    df_cancelled[(df_cancelled.Modality == y) 
                        & (df_cancelled.Scheduled < x)
                        & pd.Series(map(lambda z: datetime.date(z), 
                                        df_cancelled.Scheduled)) == datetime.date(x)].shape[0],
                                   df.Arrive, df.Modality)

##### Идеи:
- посмотреть распределение модальностей для тех ребят, которые сейчас внутри

- разница во времени между пришедшим сейчас и пришедшим перед

- предсказанное время ожидания для того, кто пришел перед текущим человеком

- зашел ли тот человек, который пришел перед тобой, в кабинете

- вышел ли тот человек, который пришел перед тобой, из кабинета

In [ ]:
df = time_of_expected(df)
df = time_of_expected_on_shedule(df)
df = time_of_procedure(df)

for i in tqdm(range(0,150,10)):
    df = len_of_queque(df,i*60)

for i in tqdm(range(0,150,10)):
    df = count_who_come_in(df,60*i)

for i in tqdm(range(0,150,10)):
    df = count_who_arrive_in(df,60*i)

for i in tqdm(range(150,301,30)):
    df = len_of_queque(df,i*60)

for i in tqdm(range(150,301,30)):
    df = count_who_come_in(df,60*i)

for i in tqdm(range(150,301,30)):
    df = count_who_arrive_in(df,60*i)
    
for i in tqdm(range(1,8)):    
    df = mean_time_of_exp(df,i,day)
    df = std_time_of_exp(df,i,day)
    df = mean_time_of_proced(df,i,day)
    df = std_time_of_proced(df,i,day)
    df = mean_time_of_exp_schel(df,i,day)
    df = std_time_of_exp_schel(df,i,day)

for i in list(df.columns[22:]):
    df[i] = df[i].fillna(0)


for i in tqdm(['Modality','Exam','day_of_week','month']):
    df,temp_uniq = dummie_train(df,i)

df['has_cancelled_today'] = has_cancelled_today(df)

In [88]:
with open('../preproc_df.pkl', 'wb') as f:
    pkl.dump(df, f)